# TP1
## Grupo 17:
**PG50315 - David Alexandre Ferreira Duarte**

**PG51247 - João Rafael Cerqueira Monteiro**

## Exercício 1. 
Use a package Criptography  para 
Criar um comunicação privada assíncrona entre um agente Emitter e um agente Receiver que cubra os seguintes aspectos:
    1. Autenticação do criptograma e dos metadados (associated data). Usar uma cifra simétrica  num modo **HMAC**  que seja seguro contra ataques aos “nounces” .
    2. Os “nounces” são gerados por um gerador pseudo aleatório (PRG) construído por um função de hash em modo XOF.
    3. O par de chaves $$cipher\_key$$, $$mac\_key$$ , para cifra e autenticação, é acordado entre agentes usando o protocolo ECDH com autenticação dos agentes usando assinaturas ECDSA.

In [1]:
!pip install cryptography
!pip install nest_asyncio

In [6]:
# Imports
import os
import asyncio
from pickle import dumps, loads
from cryptography.hazmat.primitives import hashes, hmac, serialization
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.serialization import load_pem_public_key
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.exceptions import InvalidSignature


In [9]:
def gerarChaves():
    cifraChavePrivada = ec.generate_private_key()
    cifraChavePublica = cifraChavePrivada.public_key()
    
    macChavePrivada = ec.generate_private_key()
    macChavePublica = macChavePrivada.public_key()
    
    mensagem = {'sms_cipher': cifraChavePublica.public_bytes(encoding=serialization.Encoding.PEM, format=serialization.PublicFormat.SubjectPublicKeyInfo), 
                'sms_mac': macChavePublica.public_bytes(encoding=serialization.Encoding.PEM, format= serialization.PublicFormat.SubjectPublicKeyInfo)}
    
    return dumps(mensagem), cifraChavePrivada, macChavePrivada

In [10]:
def gerarAssinatura(sms, chavePrivada):
    
    assinatura = chavePrivada.sign(sms, hashes.SHA256())
    
    smsFinal = {'message': sms, 
                'signature': assinatura, 
                'publicKey': chavePrivada.public_key().public_bytes(encoding=serialization.Encoding.PEM, format=serialization.PublicFormat.SubjectPublicKeyInfo)}
    
    return smsFinal

In [12]:
def gerarChavePartilhada(sms, cifraChavePrivada, macChavePrivada):
    
    cifraChavePublica = load_pem_public_key(sms['sms_cipher'])
    macChavePublica = load_pem_public_key(sms['sms_mac'])
    
    cifraChave = cifraChavePrivada.exchange(cifraChavePublica)
    macChave = macChavePrivada.exchange(macChavePublica)
    
    cifraChavePartlhada = HDKF(algorithm=hashes.SHA256(), length=32, salt=None, info=b'handshake data',).derive(cifraChave)
    macChavePartlhada = HDKF(algorithm=hashes.SHA256(), length=32, salt=None, info=b'handshake data',).derive(macChave)

    return cifraChavePartilhada, macChavePartlhada

In [13]:
def gerarMac(chave, crypto):
    h = hmac.HMAC(chave, hashes.SHA256(), backend=default_backend())
    h.update(crypto)
    return h.finalize()

def cifrar(plaintext, cifraChave, macChave, additionalData):
    
    digest = hashes.Hash(hashes.SHA256())
    nounce = digest.finalize()
    
    encryptor = Cipher(algorithms.AES(cifraChave), modes.GCM(nounce), backend=default_backend()).encryptor()
    
    encryptor.authenticate_additional_data(additionalData)
    
    ciphertext = encryptor.update(plaintext.encode()) + encryptor.finalize()
    
    sms = {'nounce': nounce,
           'tag': encryptor.tag,
           'cipher': ciphertext}
    
    hmac = gerarMac(macChave, dumps(sms))
    
    return {'message': sms, 'tag': hmac}

In [14]:
def decifrar(ciphertext, cifraChave, macChave, additionalData):
    
    texto = ciphertext['message']
    hmac = ciphertext['tag']
    
    macDestino = gerarMac(macChave, dumps(texto))
    
    if(hmac != macDestino):
        return 'ERRO - MAC não é igual'
    
    nounce = texto['nounce']
    tag = texto['tag']
    mensagem = texto['cipher']
    
    decryptor = Cipher(algorithms.AES(cifraChave), modes.GCM(nounce, tag), backend=default_backend()).decryptor()
    
    decryptor.authenticate_additional_data(additionalData)
    
    plaintext = decryptor.update(mensagem) + decryptor.finalize()
    
    return plaintext.decode()

In [16]:
def Emitter(conn):
    sms, cifraChavePrivada, macChavePrivada = gerarChaves()
    smsFinal = gerarAssinatura(sms)
    
    conn.send(smsFinal)
    
    sms = conn.recv()
    
    ecdsaPublico = load_pem_public_key(sms['publicKey'])
    
    try:
        ecdsaPublico.verify(sms['signature'], sms['message'], hashes.SHA256())
        
        pacoteSMS = loads(sms['message'])
        cifraChavePartilhada, macChavePartilhada = gerarChavePartilhada(pacoteSMS, cifraChavePrivada, macChavePrivada)
        
        texto = "Mensagem do emitter para o receiver"
        
        dadosAssociados = os.urandom(16)
        
        hmacChave = gerarMac(macChavePartilhada, macChavePartilhada)
        
        mensagem = cifrar(texto, cifraChavePartilhada, macChavePartilhada, dadosAssociados)
        
        mensagem['hmac_key'] = hmacChave
        mensagem['associated_data'] = dadosAssociados
        
        conn.send(mensagem)
        
    except InvalidSignature:
        print("ERRO: Mensagem não autenticada")
    
    conn.close()

In [18]:
def Receiver(conn):
    pkg, privateKey_cipher, privateKey_mac = generateKeys()
    finalPkg = generateSignature(pkg)
    #recebe a mensagem do seu lado do Pipe
    msg = conn.recv()
    print("R: Receiving public keys from emitter...")
    public_DSA = load_pem_public_key(msg['pub_key'])
    try:
    #verificar assinatura da mensagem recebida
        public_DSA.verify(msg['signature'],msg['message'],hashes.SHA256())
        print("R: The message is authentic.")
        #geração das chaves partilhadas
        pkg_msg = loads(msg['message'])
        sharedKey_cipher, sharedKey_mac =generateSharedKey(pkg_msg,privateKey_cipher,privateKey_mac)
        print("R: Sending public keys to emitter...")
        conn.send(finalPkg)
        #recebe a mensagem do seu lado do Pipe
        message = conn.recv()
        print('R: Cipher message received ')
        print(message)
        hmac_key = message['hmac_key']
        associatedData = message['associated_data']
        #verificar o código de autenticação
        if hmac_key == generateMac(sharedKey_mac,sharedKey_mac):
        #decifrar a mensagem
            print("R: Decrypting message...")
            final_message = decrypt(message, sharedKey_cipher, sharedKey_mac,associatedData)
            print("Final Message: " + final_message)
        else:
            print('ERROR - Different keys used.')
    except InvalidSignature:
        print("R: The message is not authentic.")
    conn.close()
